<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220402_sm_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.2 MB/s 
     |████████████████████████████████| 3.8 MB 82.4 MB/s 
     |████████████████████████████████| 1.1 MB 74.4 MB/s 
     |████████████████████████████████| 212 kB 87.8 MB/s 
     |████████████████████████████████| 136 kB 92.3 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 127 kB 86.5 MB/s 
     |████████████████████████████████| 144 kB 90.6 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 92.6 MB/s 
     |████████████████████████████████| 895 kB 75.8 MB/s 
     |████████████████████████████████| 596 kB 78.1 MB/s 
     |████████████████████████████████| 6.5 MB 56.1 MB/s 
     |████████████████████████████████| 1.2 MB 72.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [ ]:
from accelerate import Accelerator

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [ ]:
submission

In [ ]:
df = df.fillna('')

In [ ]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df

In [ ]:
train = df[['text', 'target']]
train

# 토크나이저, 함수

In [ ]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [ ]:
my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
train

In [ ]:
my_dict['0.0 0.0 0.0'] = len(my_dict)

In [ ]:
train[train['text'].isnull()]

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [ ]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 20
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )
  my_acc = []
  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      my_acc.append(test_acc / (batch_id+1))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220402_inference/model' + str(fold), save_function=accelerator.save)
  plt.plot(len(my_acc), my_acc)
  plt.show()

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

# 추론

In [ ]:
test = test.fillna('')

In [ ]:
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test

In [ ]:
test = test[['text', 'target']]
test

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220402_inference/model' + str(fold), num_labels=225)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220402_inference/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220402_inference/pred_prob'+str(fold)+'.csv', index=False)

In [ ]:
print(my_variable)

In [ ]:
submission

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label4.csv')

In [ ]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [ ]:
df.head(3)

In [ ]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

In [ ]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

In [ ]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob4.csv')

In [ ]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

In [ ]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

In [ ]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

In [ ]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [ ]:
soft[soft['soft'] != soft['hard']]

In [ ]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

In [ ]:
submission

In [ ]:
print(my_variable)

In [ ]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = df['soft'].map(split_label)

In [ ]:
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/220331_inference/submission.csv', index=False)